In [1]:
!pip install transformers
!pip install pytorch-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 79.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import regex as re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import math
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#import tensorflow as tf
#import tensorflow.keras.backend as K
import tokenizers
#from transformers import BertTokenizer, TFBertModel, BertForSequenceClassification, BertConfig

from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
import torch

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [4]:
"""
# Detect hardware, return appropriate distribution strategy (you can see that it is pretty easy to set up).
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print('Number of replicas:', strategy.num_replicas_in_sync)
"""

import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Running on GPU: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')
    print('Running on CPU')

# Number of GPUs available
num_gpus = torch.cuda.device_count()
print('Number of GPUs:', num_gpus)

# If using multiple GPUs, use DataParallel or DistributedDataParallel
if num_gpus > 1:
    model = torch.nn.DataParallel(model)

Running on CPU
Number of GPUs: 0


In [6]:
"""
MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 256
ARTIFACTS_PATH = '../artifacts/'

BATCH_SIZE = 8 * strategy.num_replicas_in_sync
EPOCHS = 3

if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)
"""

import torch
import os

MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 256
ARTIFACTS_PATH = '../../artifacts/'

# Determine the number of devices (GPUs/CPUs)
if torch.cuda.is_available():
    num_replicas = torch.cuda.device_count()
    print(f'Using {num_replicas} GPU(s)')
else:
    num_replicas = 1  # Use CPU or single GPU
    print('Using CPU')

# Set the batch size and number of epochs
BATCH_SIZE = 8 * num_replicas
EPOCHS = 3

# Create the artifacts directory if it doesn't exist
if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)

print(f'Model: {MODEL_NAME}, Max Length: {MAX_LEN}, Batch Size: {BATCH_SIZE}, Epochs: {EPOCHS}')

Using CPU
Model: bert-base-uncased, Max Length: 256, Batch Size: 8, Epochs: 3


In [10]:
import pandas as pd
import re
import string

import nltk
from nltk.stem.wordnet import WordNetLemmatizer


from timeit import default_timer as timer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection, svm, metrics
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, recall_score, precision_score, hamming_loss, jaccard_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups

from pprint import pprint
import argparse

import re
import string

import cufflinks as cf
import plotly.offline as pyo
import plotly.graph_objs as go

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import pickle


from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

#
# we assume everything runs from bin directory
#
PICKLE_DIR = '../../pickles/'
OUT_DIR = '../../out/'
DATASET_DIR = '../../datasets/'


#dataset_available = {'reuters21578', '20newsgroups', 'ohsumed', 'rcv1'}
dataset_available = {'20newsgroups', 'bbc-news'}

In [11]:

# --------------------------------------------------------------------------------------------------------------------------------------------

def create_confusion_matrix(y_test, y_pred, title, file_name=OUT_DIR+'svm_20newsgroups_confusion_matrix_best_model_table.png', debug=False):

    print("Creating confusion matrix...")

    # Assuming y_test and y_pred_best are already defined
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Plotting the confusion matrix as a table with numbers
    fig, ax = plt.subplots(figsize=(12, 8))  # Increase the width and height of the figure

    # Hide axes
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create the table with smaller font sizes and adjusted scale
    table = ax.table(
        cellText=conf_matrix,
        rowLabels=[f'Actual {i}' for i in range(conf_matrix.shape[0])],
        colLabels=[f'Predicted {i}' for i in range(conf_matrix.shape[1])],
        cellLoc='center',
        loc='center'
    )

    # Adjust the font size and layout
    table.auto_set_font_size(False)
    table.set_fontsize(10)  # Reduced font size for better fitting
    table.scale(1.2, 1.2)

    # Add a title with centered text
    plt.title(title, fontsize=16, pad=20)

    # Adjust layout to add more padding around the plot
    plt.subplots_adjust(left=0.2, right=0.9, top=0.9, bottom=0.1)  # Increase padding on the left

    # Save the plot to a file
    confusion_matrix_filename = file_name
    plt.savefig(confusion_matrix_filename, bbox_inches='tight')  # Ensure everything is saved in the output file
    plt.show()

    print(f"Confusion matrix saved as {confusion_matrix_filename}")

    accuracy = accuracy_score(y_test, y_pred)

    # Plain text explanation of the confusion matrix
    if debug:
        print("\nHow to read this confusion matrix:")
        print("------------------------------------------------------")
        print("The confusion matrix shows the performance of the classification model.")
        print("Each row of the matrix represents the actual classes, while each column represents the predicted classes.")
        print("Values on the diagonal (from top-left to bottom-right) represent correct predictions (true positives and true negatives).")
        print("Values outside the diagonal represent incorrect predictions (false positives and false negatives).")
        print("\nAccuracy Score: {:.2f}%".format(accuracy * 100))
        
        print("\nConfusion Matrix Values:")
        for i in range(len(conf_matrix)):
            print(f"Actual class {i}:")
            for j in range(len(conf_matrix[i])):
                print(f"  Predicted as class {j}: {conf_matrix[i][j]}")

# --------------------------------------------------------------------------------------------------------------------------------------------



# --------------------------------------------------------------------------------------------------------------
#
# Utility functions for preprocessing data
#
# --------------------------------------------------------------------------------------------------------------
def missing_values(df):
    """
    Calculate the percentage of missing values for each column in a DataFrame.
    
    Args:
    df (pd.DataFrame): The input DataFrame to analyze.
    
    Returns:
    pd.DataFrame: A DataFrame containing the total count and percentage of missing values for each column.
    """
    # Calculate total missing values and their percentage
    total = df.isnull().sum()
    percent = (total / len(df) * 100)
    
    # Create a DataFrame with the results
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    # Sort the DataFrame by percentage of missing values (descending)
    missing_data = missing_data.sort_values('Percent', ascending=False)
    
    # Filter out columns with no missing values
    missing_data = missing_data[missing_data['Total'] > 0]
    
    print("Columns with missing values:")
    print(missing_data)
    
    return missing_data


def remove_punctuation(x):
    punctuationfree="".join([i for i in x if i not in string.punctuation])
    return punctuationfree


# Function to lemmatize text with memory optimization
def lemmatization(texts, chunk_size=1000):
    lmtzr = WordNetLemmatizer()
    
    num_chunks = len(texts) // chunk_size + 1
    #print(f"Number of chunks: {num_chunks}")
    for i in range(num_chunks):
        chunk = texts[i*chunk_size:(i+1)*chunk_size]
        texts[i*chunk_size:(i+1)*chunk_size] = [' '.join([lmtzr.lemmatize(word) for word in text.split()]) for text in chunk]
    
    return texts


def preprocessDataset(train_text):
    # Ensure input is string
    train_text = str(train_text)
    
    # Word tokenization using NLTK's word_tokenize
    tokenized_train_set = word_tokenize(train_text.lower())
    
    # Stop word removal
    stop_words = set(stopwords.words('english'))
    stopwordremove = [i for i in tokenized_train_set if i not in stop_words]
    
    # Join words into sentence
    stopwordremove_text = ' '.join(stopwordremove)
    
    # Remove numbers
    numberremove_text = ''.join(c for c in stopwordremove_text if not c.isdigit())
    
    # Stemming using NLTK's PorterStemmer
    stemmer = PorterStemmer()
    stem_input = word_tokenize(numberremove_text)
    stem_text = ' '.join([stemmer.stem(word) for word in stem_input])
    
    # Lemmatization using NLTK's WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    lem_input = word_tokenize(stem_text)
    lem_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lem_input])
    
    return lem_text

# --------------------------------------------------------------------------------------------------------------


def load_data(dataset='20newsgroups'):

    print(f"Loading data set {dataset}...")

    if (dataset == '20newsgroups'):
        
        print("Loading 20 newsgroups dataset...")

        # Fetch the 20 newsgroups dataset
        newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

        # Create a DataFrame from the Bunch object
        df = pd.DataFrame({
            'text': newsgroups.data,
            'category': newsgroups.target
        })

        # Add category names
        df['category_name'] = [newsgroups.target_names[i] for i in df['category']]

        print(f"Number of documents: {len(df)}")
        print(f"Number of categories: {len(df['category'].unique())}")
        print(f"Number of category names: {len(df['category_name'].unique())}")
        #pprint(list(df.target_names))
        #pprint(list(df.category_name))

        #df = df['category'].unique()    

        missing_values_df = missing_values(df)
        print(f"missing values:", missing_values_df)

        ### Start of Text Pre-processing
        print("preproccessing...")

        nltk.download('punkt')
        nltk.download('averaged_perceptron_tagger')
        nltk.download('wordnet')
        nltk.download('omw-1.4')

        string.punctuation

        ### 2. To LowerCase

        df['CleanedText'] = (df.text.apply(lambda x: x.lower()))

        ### 3. Removing Numbers and Special Characters including XXXXXX

        df['CleanedText'] =  (df.CleanedText.apply(lambda x: re.sub('\W+', ' ', x)))
        regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')

        df['CleanedText'] =  (df.CleanedText.apply(lambda x: re.sub(regex, '', x)))
        df['CleanedText'] =  (df.CleanedText.apply(lambda x: re.sub('xxxx', '', x)))
        df['CleanedText'] =  (df.CleanedText.apply(lambda x: re.sub('xx', '', x)))

        print("removing punctuation...")

        df['CleanedText'] =  (df.CleanedText.apply(lambda x: remove_punctuation(x)))

        ### 5. Tokenization
        #data['TokenizedText'] =  (data.CleanedText.apply(lambda x: re.split('W+',x)))

        print("removing stopwords...")
        from nltk.corpus import stopwords
        nltk.download('stopwords')
        stopwords = set(stopwords.words("english"))
        df['CleanedText'] = df.CleanedText.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
        print("Stopwords removed")
        #print(df['CleanedText'][0])

        ## TFIDF already tokenizes the text so no need to tokenize it here
        # from nltk.tokenize import sent_tokenize, word_tokenize
        # data2['TokenizedText'] = data2.CleanedText.apply(word_tokenize)


        ### 7. Text Normalization  [Lemmatization] -->better than Stemming since it returns actual words
        ## lemmatization is an intelligent operation that uses dictionaries

        print("Lemmatizing...")

        df['LemmatizedText'] = lemmatization(df['CleanedText'])

        print("Lemmatized")
        #print(df['CleanedText'][0])
        #print(df['LemmatizedText'][0])

        print("Tokenizing...")
        
        # Tokenize the text data
        df['tokenized'] = df['text'].str.lower().apply(nltk.word_tokenize)

        #return df, df['category'].unique()
        return df
    
    elif (dataset == 'bbc-news'):

        print("Loading BBC News dataset...")

        for dirname, _, filenames in os.walk(DATASET_DIR+'bbc-news'):
            for filename in filenames:
                print(os.path.join(dirname, filename))

        train_set = pd.read_csv(DATASET_DIR+'bbc-news/BBC News Train.csv')
        test_set = pd.read_csv(DATASET_DIR+'bbc-news/BBC News Test.csv')

        print("train_set:", train_set.shape)
        print(train_set.head())
        print("test_set:", test_set.shape)
        print(test_set.head())

        target_category = train_set['Category'].unique()
        print("target categories:", target_category)

        train_set['categoryId'] = train_set['Category'].factorize()[0]
        
        category = train_set[["Category","categoryId"]].drop_duplicates().sort_values('categoryId')
        print("after de-duping:", category)

        print(train_set.groupby('Category').categoryId.count())

        text = train_set["Text"] 
        print("text:\n", text.head())

        category = train_set["Category"]
        print("categories:\n", category.head())

        print("preprocessing...")
        train_set['Text'] = train_set['Text'].apply(preprocessDataset)
        text = train_set['Text']
        category = train_set['Category']
        print("text:\n", type(text), text.shape)
        print(text.head())

        #return text, category
        return train_set
    else:
        print(f"Dataset '{dataset}' not available.")
        return None


# --------------------------------------------------------------------------------------------------------------
# Core processing function
# --------------------------------------------------------------------------------------------------------------
def classify(dataset='20newsgrouops', args=None):
    
    if (args is None):
        print("No arguments passed.")
        return
    
    if (args.dataset not in dataset_available):
        print(f"Dataset '{args.dataset}' not available.")
        return
    
    pickle_file_name=f'{dataset}_{args.mode}_tokenized.pickle'

    print(f"Classifying {dataset}...")

    print(f"Loading data set {dataset}...")

    # Define the path to the pickle file
    pickle_file = PICKLE_DIR + pickle_file_name

    if os.path.exists(pickle_file):                                         # if the pickle file exists
        
        print(f"Loading tokenized data from '{pickle_file}'...")
        
        if (dataset == '20newsgroups'):
            # Initialize an empty DataFrame with the desired columns
            columns = ['tokenized', 'CleanedText', 'LemmatizedText', 'category', 'category_name', 'text']
            
            df = pd.DataFrame(columns=columns)
            
            # Load the data from the pickle file into the DataFrame
            with open(pickle_file, 'rb') as f:
                df = pickle.load(f)

            #categories = df['category'].unique()
            
        elif (dataset == 'bbc-news'):

            columns = []

            df = pd.DataFrame(columns=columns)
            
            with open(pickle_file, 'rb') as f:
                df = pickle.load(f)
    else:
        print(f"'{pickle_file}' not found, retrieving and preprocessing data set {dataset}...")

        #df, categories = load_data(dataset)
        df = load_data(dataset)

        print("df:", df.shape)
        print(df.head())

        # Save the tokenized DataFrame to a pickle file
        if (dataset == '20newsgroups'):
            with open(pickle_file, 'wb') as f:
                pickle.dump(df[['tokenized', 'CleanedText', 'LemmatizedText', 'category', 'category_name', 'text']], f)

        elif (dataset == 'bbc-news'):
            with open(pickle_file, 'wb') as f:
                pickle.dump(df[['ArticleId', 'Text', 'Category']], f)
                #pickle.dump(df, f)

    print("Tokenized data loaded, df:", df.shape)
    print(df.head())
    #print("categories:", categories)

    print("Processing data...")

    if (dataset == '20newsgroups'):

        print("classifying 20 newsgroups...")

        """
        # POS Tagging and Counting
        tagged_titles = df['text'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))

        def count_tags(title_with_tags):
            tag_count = {}
            for word, tag in title_with_tags:
                tag_count[tag] = tag_count.get(tag, 0) + 1
            return tag_count

        # Create a DataFrame with POS tag counts
        tagged_titles_df = pd.DataFrame(tagged_titles.apply(lambda x: count_tags(x)).tolist()).fillna(0)

        # Sum the occurrences of each tag across all documents
        tagged_titles_sum = tagged_titles_df.sum().sort_values(ascending=False)

        # Plot POS Tag Frequency
        trace = go.Bar(x=tagged_titles_sum.index, y=tagged_titles_sum.values)
        layout = go.Layout(title='Frequency of POS Tags in IT Support Tickets Dataset', xaxis=dict(title='POS'), yaxis=dict(title='Count'))
        fig = go.Figure(data=[trace], layout=layout)

        # This will open the plot in the default web browser
        pyo.plot(fig, filename='../../out/pos_tag_frequency.html')
        """

        # Feature Extraction and Model Training
        print("Splitting the dataset...")

        X = df['CleanedText']
        y = df['category']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

        print("X_train:", type(X_train), X_train.shape)
        print("X_test:", type(X_test), X_test.shape)
        
        print("Y_train:", type(y_train), y_train.shape)
        print("Y_test:", type(y_test), y_test.shape)

        print("Running model...")

        run_model(X_train, X_test, y_train, y_test, args)
    
    elif (dataset == 'bbc-news'):

        print(f'classifying bbc-news...')
        
        X_train, X_test, y_train, y_test = train_test_split(
            df['Text'],
            df['Category'], 
            test_size = 0.3, 
            random_state = 60,
            shuffle=True, 
            stratify=df['Category']
            )

        print("X_train:", type(X_train), X_train.shape)
        print("X_test:", type(X_test), X_test.shape)
        
        print("Y_train:", type(y_train), y_train.shape)
        print("Y_test:", type(y_test), y_test.shape)

        print("Running model...")

        run_model(X_train, X_test, y_train, y_test, args)

# -------------------------------------------------------------------------------------------------------------------------------------------------

def run_svm_model(X_train, X_test, y_train, y_test, args):

    print("Training default Support Vector Machine model...")
    
    default_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('lr', LinearSVC(max_iter=1000))
    ])

    default_pipeline.fit(X_train, y_train)
    y_pred_default = default_pipeline.predict(X_test)

    print("\nDefault Support Vector Mechine Model Performance:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_default):.4f}")
    print(classification_report(y_true=y_test, y_pred=y_pred_default, digits=4))

    if (args.optimc):

        # Optimize Support Vector Machine with GridSearchCV
        print("Optimizing Support Vector Machine model with GridSearchCV...")

        # Define the pipeline
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('svm', LinearSVC(max_iter=1000))
        ])

        # Define the parameter grid
        param_grid = {
            'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],     # Unigrams, bigrams, or trigrams
            'tfidf__use_idf': [True, False],                    # Whether to use IDF
            'tfidf__sublinear_tf': [True, False],               # Sublinear term frequency
            'svm__penalty': ['l1', 'l2'],                       # Regularization method
            'svm__loss': ['hinge', 'squared_hinge'],            # Loss function
            'svm__multi_class': ['ovr', 'crammer_singer'],      # Multi-class strategy
            'svm__class_weight': [None, 'balanced'],            # Class weights
            'svm__C': np.logspace(-3, 3, 7)                     # Regularization parameter   
        }

        print("param_grid:", param_grid)

        cross_validation = StratifiedKFold()

        scorers = {
            'accuracy_score': make_scorer(accuracy_score),
            'f1_score': make_scorer(f1_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'hamming_loss': make_scorer(hamming_loss),
            'jaccard_score': make_scorer(jaccard_score, average='micro')
            }

        grid_search = GridSearchCV(
            n_jobs=-1, 
            estimator=pipeline,
            refit='f1_score',
            param_grid=param_grid,
            cv=cross_validation,
            #scoring=scoring
            scoring=scorers,
            return_train_score=True         # ensure train scores are calculated
            )

        # Fit the model
        grid_search.fit(X_train, y_train)

        print('Best parameters: {}'.format(grid_search.best_params_))
        print("best_estimator:", grid_search.best_estimator_)
        print('Best score: {}'.format(grid_search.best_score_))
        print("cv_results_:", grid_search.cv_results_)

        results = grid_search.cv_results_

        if (args.plot):

            print("Plotting the results...")

            # Define the metrics we want to plot
            metrics_to_plot = ['accuracy_score', 'f1_score', 'recall_score', 'precision_score', 'hamming_loss']

            # Iterate over each metric to create a separate plot
            for metric in metrics_to_plot:
                traces = []

                print(f"Plotting {metric}...")

                for sample in ["train", "test"]:

                    key_mean = f"mean_{sample}_{metric}"
                    key_std = f"std_{sample}_{metric}"

                    print(f"Plotting {key_mean}...")
                    print(f"Plotting {key_std}...")

                    # Directly use the keys without conditional check
                    sample_score_mean = np.nan_to_num(np.array(results[key_mean]) * 100)  # Convert to percentage and handle NaN
                    sample_score_std = np.nan_to_num(np.array(results[key_std]) * 100)  # Convert to percentage and handle NaN

                    x_axis = np.linspace(0, 100, len(sample_score_mean))

                    # Create the trace for Plotly
                    traces.append(
                        go.Scatter(
                            x=x_axis,
                            y=sample_score_mean,
                            mode='lines+markers',
                            name=f"{metric} ({sample})",
                            line=dict(dash='dash' if sample == 'train' else 'solid'),
                            error_y=dict(
                                type='data',
                                array=sample_score_std,
                                visible=True
                            ),
                            hoverinfo='x+y+name'
                        )
                    )

                # Define the layout of the plot
                layout = go.Layout(
                    title={'text': f"Training and Test Scores for {metric.capitalize()}",
                        'y':0.9,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    xaxis=dict(title="Training Sample Percentage (%)"),
                    yaxis=dict(title="Score (%)", range=[0, 100]),
                    hovermode='closest'
                )

                # Create the figure
                fig = go.Figure(data=traces, layout=layout)

                # Write the plot to an HTML file
                filename = f'{OUT_DIR}training_test_scores_{metric}.html'
                pyo.plot(fig, filename=filename)

                print(f"Saved plot for {metric} as {filename}")

        # Extract the best estimator from the GridSearchCV
        best_model = grid_search.best_estimator_

        # Predict on the test set using the best model
        y_pred_best = best_model.predict(X_test)

        print("Accuracy best score:", metrics.accuracy_score(y_test, y_pred_best))
        print(classification_report(y_true=y_test, y_pred=y_pred_best, digits=4))


def run_lr_model(X_train, X_test, y_train, y_test, args):

    # Default Logistic Regression Model
    print("Training default Logistic Regression model...")
    default_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('lr', LogisticRegression(max_iter=1000))
    ])

    default_pipeline.fit(X_train, y_train)
    y_pred_default = default_pipeline.predict(X_test)

    print("\nDefault Logistic Regression Model Performance:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_default):.4f}")
    print(classification_report(y_true=y_test, y_pred=y_pred_default, digits=4))

    if (args.optimc):
        # Optimize Logistic Regression with GridSearchCV
        print("Optimizing Logistic Regression model with GridSearchCV...")

        # Define the pipeline
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('lr', LogisticRegression(max_iter=1000))
        ])

        # Define the parameter grid
        param_grid = {
            'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],     # Unigrams, bigrams, or trigrams
            'tfidf__use_idf': [True, False],                    # Whether to use IDF
            'tfidf__sublinear_tf': [True, False],               # Sublinear term frequency
            'lr__C': [0.01, 0.1, 1, 10, 100],                   # Inverse of regularization strength
            'lr__penalty': ['l2'],                              # Regularization method (L2 Ridge)
            'lr__solver': ['liblinear', 'lbfgs']                # Solver types
        }

        print("param_grid:", param_grid)

        # Define scorers
        scorers = {
            'accuracy_score': make_scorer(accuracy_score),
            'f1_score': make_scorer(f1_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro')
        }

        # Initialize GridSearchCV
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=scorers,
            refit='f1_score',  # Optimize on F1 Score
            cv=StratifiedKFold(n_splits=5),
            n_jobs=-1,
            return_train_score=True
        )

        # Fit the model
        grid_search.fit(X_train, y_train)

        # Display the best parameters
        print('Best parameters found by GridSearchCV:')
        print(grid_search.best_params_)

        # Evaluate on the test set
        y_pred_optimized = grid_search.best_estimator_.predict(X_test)

        print("\nOptimized Logistic Regression Model Performance:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred_optimized):.4f}")
        print(classification_report(y_true=y_test, y_pred=y_pred_optimized, digits=4))

    if (args.cm):
        # Optionally, plot confusion matrix for the optimized model
        create_confusion_matrix(
            y_test, 
            y_pred_optimized, 
            title='Confusion Matrix for Optimized Logistic Regression Model',
            file_name=OUT_DIR+'bbc_news_logistic_regression_confusion_matrix.png',
            debug=False
        )


def run_nb_model(X_train, X_test, y_train, y_test, args):

    print("Building default Naive Bayes Classifier...")

    nb = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB())
        ])
    
    nb.fit(X_train,y_train)

    test_predict = nb.predict(X_test)

    train_accuracy = round(nb.score(X_train,y_train)*100)
    test_accuracy =round(accuracy_score(test_predict, y_test)*100)

    print("Naive Bayes Train Accuracy Score : {}% ".format(train_accuracy ))
    print("Naive Bayes Test Accuracy Score  : {}% ".format(test_accuracy ))
    print(classification_report(y_true=test_predict, y_pred=y_test, digits=4))

    if (args.optimc):

        print("Optimizing the model using GridSearchCV...")

        # Define a pipeline
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('nb', MultinomialNB())
        ])

        # Define the parameter grid
        param_grid = {
            'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],         # Unigrams, bigrams, or trigrams
            'tfidf__use_idf': [True, False],                        # Whether to use IDF
            'tfidf__sublinear_tf': [True, False],                   # Sublinear term frequency
            'nb__alpha': [0.1, 0.5, 1.0, 1.5, 2.0],                 # Smoothing parameter for Naive Bayes
        }

        print("param_grid:", param_grid)

        # Define scorers
        scorers = {
            'accuracy_score': make_scorer(accuracy_score),
            'f1_score': make_scorer(f1_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'hamming_loss': make_scorer(hamming_loss),
            'jaccard_score': make_scorer(jaccard_score, average='micro')
        }

        # Initialize GridSearchCV
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=scorers,
            refit='f1_score',                           # Optimize on F1 Score
            cv=StratifiedKFold(n_splits=5),
            n_jobs=-1,
            return_train_score=True
        )

        # Fit the model
        grid_search.fit(X_train, y_train)

        # Display the best parameters
        print('Best parameters found by GridSearchCV:')
        print(grid_search.best_params_)

        # Evaluate on the test set
        y_pred = grid_search.best_estimator_.predict(X_test)

        print("\nBest Estimator's Test Set Performance:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"F1 Score: {f1_score(y_test, y_pred, average='micro'):.4f}")
        print(f"Recall: {recall_score(y_test, y_pred, average='micro'):.4f}")
        print(f"Precision: {precision_score(y_test, y_pred, average='micro'):.4f}")
        print(classification_report(y_true=y_test, y_pred=y_pred, digits=4))

        if (args.cm):
            # Optionally, plot confusion matrix
            create_confusion_matrix(
                y_test, 
                y_pred, 
                title='Confusion Matrix for Optimized Naive Bayes Model',
                file_name=OUT_DIR+'bbc_news_naive_bayes_confusion_matrix.png',
                debug=False
            )


def run_model(X_train, X_test, y_train, y_test, args):

    print("Running model...")

    # Support Vector Machine Classifier
    if (args.learner == 'svm'):
        run_svm_model(X_train, X_test, y_train, y_test, args)

    # Logistic Regression Classifier
    elif (args.learner == 'lr'):
        run_lr_model(X_train, X_test, y_train, y_test, args)

    # Naive Bayes (MultinomialNB) Classifier
    elif (args.learner == 'nb'):
        run_nb_model(X_train, X_test, y_train, y_test, args)

    elif (args.learner == 'dt'):
        print("Decision Tree Classifier")
        dt = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('dt', DecisionTreeClassifier())
            ])

        dt.fit(X_train, y_train)

        test_predict = dt.predict(X_test)

        train_accuracy = round(dt.score(X_train, y_train)*100)
        test_accuracy =round(accuracy_score(test_predict, y_test)*100)

        print("Decision Tree Train Accuracy Score : {}% ".format(train_accuracy ))
        print("Decision Tree Test Accuracy Score  : {}% ".format(test_accuracy ))
        print(classification_report(y_true=test_predict, y_pred=y_test, digits=4))

    elif (args.learner == 'rf'):

        print("Random Forest Classifier")
        rfc = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('rfc', RandomForestClassifier(n_estimators=100))
            ])

        rfc.fit(X_train, y_train)

        test_predict = rfc.predict(X_test)

        train_accuracy = round(rfc.score(X_train, y_train)*100)
        test_accuracy =round(accuracy_score(test_predict, y_test)*100)

        print("K-Nearest Neighbour Train Accuracy Score : {}% ".format(train_accuracy ))
        print("K-Nearest Neighbour Test Accuracy Score  : {}% ".format(test_accuracy ))
        print(classification_report(y_true=test_predict, y_pred=y_test, digits=4))

    else:
        print(f"Invalid learner '{args.learner}'")
        return


In [13]:
data = load_data('bbc-news')

Loading data set bbc-news...
Loading BBC News dataset...
../../datasets/bbc-news/BBC News Train.csv
../../datasets/bbc-news/.DS_Store
../../datasets/bbc-news/BBC News Test.csv
../../datasets/bbc-news/BBC News Sample Solution.csv
train_set: (1490, 3)
   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business
test_set: (735, 2)
   ArticleId                                               Text
0       1018  qpr keeper day heads for preston queens park r...
1       1319  software watching while you work software that...
2       1138  d arcy injury adds to ireland woe gordon d arc...
3        459  india s reliance 

In [ ]:
! gdown --id 1LxH5sC6AdrwzClPEGUAV2HVcPjWIPmc8  ##SupportTickets
#! gdown --id 1etLBrBTdokVHIuaxEmr1Koacos4IbMZL ## ConsumerComplaints

Downloading...
From: https://drive.google.com/uc?id=1LxH5sC6AdrwzClPEGUAV2HVcPjWIPmc8
To: /content/FinalFinalTwoTierSnowmirror.csv
100% 101M/101M [00:00<00:00, 217MB/s] 


In [ ]:
import pandas as pd
import io
data = pd.read_csv('FinalFinalTwoTierSnowmirror.csv')

In [ ]:
##  Creating a new Dataframe with the cols we're interested In
cols = ['FinalText', 'DV_CATEGORY']
#cols = ['TokenizedText', 'Product']
data = data[cols]
data.head(10)

,FinalText,DV_CATEGORY
0,mass data migration device rental invoice,Project Office
1,ibm cloud backup non successful hello use evau...,Compute
2,accounting request pro rate credit,Project Office
3,feature code purchase subscription receive sen...,Project Office
4,subscription account link ibm cloud bluemix si...,Project Office
5,virtual server high customer like replicate ib...,Compute
6,incorrectly early provision due error replace ...,Project Office
7,security compliance require vmware answer impl...,Compute
8,service cancellation,3rd Party Reseller
9,bluemix asset monitor compose postgresql,Platform / Console


In [ ]:
data.DV_CATEGORY.unique()

array(['Project Office', 'Compute', '3rd Party Reseller',
       'Platform / Console', 'Services', 'VPC', 'Apps',
       'Security and Identity', 'Networking', 'Integration',
       'Project Office (internal)', 'Storage'], dtype=object)

In [ ]:
X_data = data[['FinalText']].to_numpy().reshape(-1)
y_data = data[['DV_CATEGORY']].to_numpy().reshape(-1)

In [ ]:
def bert_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(str(text))

        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])

        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN

        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')

        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
# Transform categories into numbers
category_to_id = {}
category_to_name = {}

for index, c in enumerate(y_data):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name[category_id] = c

    y_data[index] = category_id

# Display dictionary
category_to_name

{0: 'Project Office',
 1: 'Compute',
 2: '3rd Party Reseller',
 3: 'Platform / Console',
 4: 'Services',
 5: 'VPC',
 6: 'Apps',
 7: 'Security and Identity',
 8: 'Networking',
 9: 'Integration',
 10: 'Project Office (internal)',
 11: 'Storage'}

In [ ]:
categories = data['DV_CATEGORY'].unique()
n_categories = len(categories)

In [ ]:
n_categories

12

In [ ]:
# Split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=777) # random_state to reproduce results

In [ ]:
X_train

array(['increase upgrade bandwidth option sale agent hello need set server account expect consumption last month local ibm',
       'make lite account get confirmation mail hour address trial ibm must spam box think proceed',
       'user provide create dev environment access name hasan mohammad regard organization space resource field',
       ..., 'test ticket ignore',
       'confirmation behavior auto scale hello support team use node red like ask question automatically start instance number alive become low minimal least confirm scaling answer yes long take since',
       'request survey charge san provide usage report something check content renewal order total balance platform support month reflect amount overage investigate whether follow invoice correct'],
      dtype=object)

In [ ]:
# Import tokenizer from HuggingFace
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
X_train = bert_encode(X_train, tokenizer)
X_test = bert_encode(X_test, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [ ]:
def build_model(n_categories):
    with strategy.scope():
        input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
        input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
        input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

        # Import BERT model from HuggingFace
        bert_model = TFBertModel.from_pretrained(MODEL_NAME)
        x = bert_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

        # Huggingface transformers have multiple outputs, embeddings are the first one,
        # so let's slice out the first position
        x = x[0]

        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

        model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr=1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
with strategy.scope():
    model = build_model(n_categories)
    model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   TFBaseModelOutputWit 109482240   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [ ]:
with strategy.scope():
    print('Training...')
    history = model.fit(X_train,
                        y_train,
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        validation_data=(X_test, y_test))

Training...
Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None,) dtype=int32>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None,) dtype=int32>]


2128/2128 [==============================] - ETA: 0s - loss: 0.8341 - accuracy: 0.7366

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None,) dtype=int32>]


2128/2128 [==============================] - 508s 199ms/step - loss: 0.8341 - accuracy: 0.7366 - val_loss: 0.7145 - val_accuracy: 0.7733
Epoch 2/3
2128/2128 [==============================] - 355s 167ms/step - loss: 0.6804 - accuracy: 0.7811 - val_loss: 0.6705 - val_accuracy: 0.7896
Epoch 3/3
2128/2128 [==============================] - 356s 167ms/step - loss: 0.6180 - accuracy: 0.7988 - val_loss: 0.6590 - val_accuracy: 0.7917


In [ ]:
y_pred

array([[0.9132952 , 0.08670478],
       [0.09615043, 0.90384954],
       [0.90086865, 0.09913131],
       [0.91933906, 0.08066092],
       [0.12343215, 0.87656784],
       [0.8872177 , 0.11278225],
       [0.16934107, 0.830659  ],
       [0.90292263, 0.09707733],
       [0.04330614, 0.9566938 ],
       [0.9645212 , 0.03547875],
       [0.95503354, 0.04496641],
       [0.8570329 , 0.14296713],
       [0.06058738, 0.9394126 ],
       [0.8674663 , 0.13253374],
       [0.27896786, 0.72103214],
       [0.09123947, 0.9087604 ],
       [0.92922574, 0.07077418],
       [0.8877487 , 0.11225132],
       [0.26868767, 0.73131233],
       [0.06149247, 0.93850744],
       [0.85076886, 0.14923121],
       [0.2576749 , 0.7423251 ],
       [0.03756809, 0.96243197],
       [0.76388615, 0.23611389],
       [0.03847211, 0.9615279 ],
       [0.9822324 , 0.01776751],
       [0.04295966, 0.95704037],
       [0.83125776, 0.1687422 ],
       [0.88538367, 0.11461624],
       [0.8722662 , 0.12773384],
       [0.